# 🏨 Partie 3 : Scraping des Données Hôtelières

## 🎯 Objectif
Extraire la liste des hôtels, leurs notes, descriptions et liens pour chaque ville cible via le site Booking.com.

## 🛠 Choix Techniques : "Requests" vs "Selenium"
Le scraping de Booking.com est complexe (protection anti-bot, chargement asynchrone). Au lieu d'utiliser **Selenium** (lourd et lent), j'ai opté pour une approche **`requests` optimisée** :
1.  **Gestion de Session** : Maintien des cookies pour simuler un utilisateur réel.
2.  **Rotation d'User-Agent** : Changement d'identité (Chrome, Firefox, Safari) à chaque tentative.
3.  **Algorithme de Réessai (Retry Logic)** : Gestion automatique des erreurs 202 (chargement) et des blocages temporaires. Le script patiente et réessaie intelligemment avant d'abandonner.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random
import re

# Chargement des villes
df_cities = pd.read_csv('cities_coordinates.csv')
hotels_data = []

# Liste de User-Agents pour la rotation (Anti-Bot)
user_agents_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15"
]

print("🏨 Démarrage du scraping Booking.com (Mode Robuste)...")

for index, row in df_cities.iterrows():
    city = row['city']
    print(f"\n🌍 [{index+1}/35] Recherche pour : {city}...")
    
    url = "https://www.booking.com/searchresults.fr.html"
    params = {
        "ss": city,
        "checkin_year": "2025", "checkin_month": "6", "checkin_monthday": "15",
        "checkout_year": "2025", "checkout_month": "6", "checkout_monthday": "16",
        "group_adults": "2", "no_rooms": "1", "group_children": "0"
    }

    # --- RETRY LOGIC (BOUCLE DE RÉESSAI) ---
    max_retries = 5
    attempt = 0
    success = False
    
    while attempt < max_retries and not success:
        attempt += 1
        
        # Rotation d'identité
        current_ua = random.choice(user_agents_list)
        headers = {
            "User-Agent": current_ua,
            "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
            "Cache-Control": "no-cache",
            "Pragma": "no-cache"
        }
        
        try:
            if attempt > 1:
                print(f"   🔄 Tentative {attempt}/{max_retries}...")

            response = requests.get(url, params=params, headers=headers, timeout=15)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                hotel_cards = soup.find_all('div', attrs={"data-testid": "property-card"})
                
                if len(hotel_cards) > 0:
                    success = True
                    print(f"   ✅ Succès ! {len(hotel_cards)} hôtels trouvés.")
                    
                    count = 0
                    for card in hotel_cards:
                        if count >= 20: break # Top 20 seulement
                        
                        # Extraction des champs
                        name_el = card.find('div', attrs={"data-testid": "title"})
                        name = name_el.text.strip() if name_el else "Inconnu"
                        
                        link_el = card.find('a', attrs={"data-testid": "title-link"})
                        link = link_el['href'] if link_el else ""
                        
                        score_el = card.find('div', attrs={"data-testid": "review-score"})
                        score_text = score_el.text.strip() if score_el else ""
                        match = re.search(r"(\d+[.,]\d+)", score_text)
                        score = match.group(1).replace(',', '.') if match else None
                        
                        desc_el = card.find('div', attrs={"data-testid": "recommended-units"})
                        description = desc_el.text.strip() if desc_el else "Pas de description"

                        hotels_data.append({
                            "city": city,
                            "hotel_name": name,
                            "url": link,
                            "score": score,
                            "description": description
                        })
                        count += 1
                else:
                    # Page vide (soft ban ou erreur de chargement)
                    pass 
            
            elif response.status_code == 202:
                print("   ⏳ Booking calcule (Code 202)...")
                
        except Exception as e:
            print(f"   ❌ Erreur technique : {e}")

        # Attente avant réessai (si échec)
        if not success and attempt < max_retries:
            time.sleep(random.uniform(5, 8))
            
    if not success:
        print(f"   💀 Abandon pour {city} après {max_retries} essais.")
    
    # Pause longue entre chaque ville
    time.sleep(random.uniform(5, 15))

# Sauvegarde
df_hotels = pd.DataFrame(hotels_data)
df_hotels.to_csv('hotels_data.csv', index=False)

print("\n" + "="*30)
print(f"🎉 Terminé ! {len(df_hotels)} hôtels scrapés.")
print("💾 Sauvegardé dans 'hotels_data.csv'")

🏨 Démarrage du scraping avec RÉESSAIS AUTOMATIQUES...

🌍 [1/35] Traitement de : Mont Saint Michel...
   ⏳ Booking calcule (Code 202)...
   💤 Echec. Attente de 5.7s avant de réesayer Mont Saint Michel...
   🔄 Tentative 2/5 avec un nouveau User-Agent...
   ✅ Succès ! 25 hôtels trouvés.
⏸️ Pause navigation de 25.0s...

🌍 [2/35] Traitement de : St Malo...
   ⏳ Booking calcule (Code 202)...
   💤 Echec. Attente de 5.4s avant de réesayer St Malo...
   🔄 Tentative 2/5 avec un nouveau User-Agent...
   ⏳ Booking calcule (Code 202)...
   💤 Echec. Attente de 9.1s avant de réesayer St Malo...
   🔄 Tentative 3/5 avec un nouveau User-Agent...
   ✅ Succès ! 25 hôtels trouvés.
⏸️ Pause navigation de 15.3s...

🌍 [3/35] Traitement de : Bayeux...
   ⏳ Booking calcule (Code 202)...
   💤 Echec. Attente de 5.6s avant de réesayer Bayeux...
   🔄 Tentative 2/5 avec un nouveau User-Agent...
   ✅ Succès ! 25 hôtels trouvés.
⏸️ Pause navigation de 7.4s...

🌍 [4/35] Traitement de : Le Havre...
   ⏳ Booking calcule 